<a href="https://colab.research.google.com/github/harishnk28/TransferLearningFineTuning/blob/main/VGGFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Write a python code to implement the VGG 16  

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
def visualize_data(images, categories, class_names):
    fig = plt.figure(figsize=(14, 6))
    fig.patch.set_facecolor('white')
    for i in range(3 * 7):
        plt.subplot(3, 7, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(images[i])
        class_index = categories[i].argmax()
        plt.xlabel(class_names[class_index])
    plt.show()

# Dataset as any image dataset but here you have to add the data augmentation (Eg: 10000images then make as 15000images using data augmentation scheme). 

In [ ]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
#Keras library for CIFAR dataset
from keras.datasets import cifar10

(x_train, y_train),(x_test, y_test)=cifar10.load_data()

#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

#Dimension of the CIFAR10 dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

170500096/170498071 [==============================] - 3s 0us/step
((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))
((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


In [ ]:
from tensorflow.keras.applications import vgg16

# Init the VGG model 
vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

58892288/58889256 [==============================] - 0s 0us/step


# Fine Tuning

In [ ]:
# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fdf92f3bed0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fdf92f4b050> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fdf8ab68e10> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fdf8a870f50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fdf8a878550> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fdf8b724050> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fdf8a883190> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fdf8a890950> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fdf8a88c8d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fdf8a896dd0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7fdf8a822990> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

# Include batch normalization at the anyplace followed by first convolution  

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)
# Create the model
model = Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)
model.add(Flatten())
model.add(Dense(256, use_bias=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

In [ ]:
#Defining the parameters
batch_size= 100
epochs=100
learn_rate=.001

In [ ]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5) 

In [ ]:
#Training the model
model.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = 10, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = 250, callbacks = [lrr], verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
350/350 [==============================] - 545s 2s/step - loss: 0.3307 - accuracy: 0.9000 - val_loss: 0.3275 - val_accuracy: 0.9000
Epoch 2/10
350/350 [==============================] - 545s 2s/step - loss: 0.3303 - accuracy: 0.9000 - val_loss: 0.3273 - val_accuracy: 0.9000
Epoch 3/10
350/350 [==============================] - 543s 2s/step - loss: 0.3300 - accuracy: 0.9000 - val_loss: 0.3272 - val_accuracy: 0.9000
Epoch 4/10
350/350 [==============================] - 543s 2s/step - loss: 0.3297 - accuracy: 0.9000 - val_loss: 0.3271 - val_accuracy: 0.9000
Epoch 5/10
350/350 [==============================] - 540s 2s/step - loss: 0.3294 - accuracy: 0.9000 - val_loss: 0.3269 - val_accuracy: 0.9000
Epoch 6/10
350/350 [==============================] - 543s 2s/step - loss: 0.3293 - accuracy: 0.9000 - val_loss: 0.3268 - val_accuracy: 0.9000
Epoch 7/10
350/350 [==============================] - 543s 2s/step - loss: 0.3290 - accuracy: 0.9000 - val_loss: 0.3267 - val_accuracy: 0.9000